In [1]:
from howlongtobeatpy import HowLongToBeat
import re
import pandas as pd
import numpy as np

In [2]:
#results = HowLongToBeat().search("Chrono Trigger")
#um = results[0]

"""
 self.game_id = -1
        # Default Game Name
        self.game_name = None

        # Alias for the same game, as a second name
        self.game_alias = None

        # The type of entry, usually "game" or "dlc"
        self.game_type = None
        self.game_image_url = None

        # Link to How Long To Beat
        self.game_web_link = None

        # The review score
        self.review_score = None

        # The name of the dev
        self.profile_dev = None

        # The list of the platforms for the title
        self.profile_platforms = None

        # Should contain the release year
        self.release_world = None

        # Similarity between this entry game name and the searched string
        # Calculated as the max similarity between the searched string and game_name & game_alias
        self.similarity = -1

        # JSON Fields
        # Full JSON response
        self.json_content = None

        # The four different completion times, in Hours
        # Remember not all games will have them valid (like coop or pvp games)
        # Main Story
        self.main_story = None

        # Main + Extra
        self.main_extra = None

        # Main Completionist
        self.completionist = None

        # All styles
        self.all_styles = None
        

"""

In [3]:
# Nome do arquivo HTML
arquivo_html = "teste.html"

# Lista para armazenar as linhas filtradas
linhas_filtradas = []

# Abre o arquivo e lê linha por linha
with open(arquivo_html, "r", encoding="utf-8") as f:
    for linha in f:
        if '<h6><a href="https://jrpgc.com/games' in linha:
            linhas_filtradas.append(linha.strip())

# Expressão regular para capturar o conteúdo entre <a> e </a>
padrao = re.compile(r'<a [^>]+>(.*?)</a>')

# Extrair o texto das tags <a>
titulos = [match.group(1) for linha in linhas_filtradas if (match := padrao.search(linha))]

In [4]:
len(titulos)

3185

In [5]:
#results = HowLongToBeat().search("digital devil story II", similarity_case_sensitive=False)

In [6]:
#for result in results:
#    print(result.game_name)
#    print(result.similarity)

In [7]:
#results[0].game_name

In [8]:
#for i in titulos:
#    if "Digital Devil Story" in i:
#        print(i)

jogos = pd.DataFrame()
for titulo in titulos:
    results = HowLongToBeat().search(titulo, similarity_case_sensitive=False)
    if results is not None and len(results) > 0:
        best_element = max(results, key=lambda element: element.similarity)
        jogos.append(best_element.json_content)
        



    

In [9]:
titulos_all = titulos

In [10]:
"""import asyncio
import pandas as pd
from howlongtobeatpy import HowLongToBeat

async def buscar_jogo(titulo):
    results = await HowLongToBeat().async_search(titulo, similarity_case_sensitive=False)
    if results is not None and len(results) > 0:
        best_element = max(results, key=lambda element: element.similarity)
        return best_element.json_content
    return None

async def main():
    jogos = []

    # Executa todas as buscas em paralelo
    resultados = await asyncio.gather(*(buscar_jogo(titulo) for titulo in titulos))

    # Filtra os resultados válidos e cria o DataFrame
    jogos = pd.DataFrame([r for r in resultados if r is not None])
    jogos.to_parquet("jrpgs_db.parquet", engine="pyarrow")
    print(jogos)

# Rodar a função assíncrona
#asyncio.run(main())
await main()
"""

'import asyncio\nimport pandas as pd\nfrom howlongtobeatpy import HowLongToBeat\n\nasync def buscar_jogo(titulo):\n    results = await HowLongToBeat().async_search(titulo, similarity_case_sensitive=False)\n    if results is not None and len(results) > 0:\n        best_element = max(results, key=lambda element: element.similarity)\n        return best_element.json_content\n    return None\n\nasync def main():\n    jogos = []\n\n    # Executa todas as buscas em paralelo\n    resultados = await asyncio.gather(*(buscar_jogo(titulo) for titulo in titulos))\n\n    # Filtra os resultados válidos e cria o DataFrame\n    jogos = pd.DataFrame([r for r in resultados if r is not None])\n    jogos.to_parquet("jrpgs_db.parquet", engine="pyarrow")\n    print(jogos)\n\n# Rodar a função assíncrona\n#asyncio.run(main())\nawait main()\n'

In [15]:
import asyncio
import pandas as pd
from howlongtobeatpy import HowLongToBeat
import aiohttp

lista_jsons = []

async def buscar_jogo(titulo, session):
    """Busca informações sobre um jogo na API do HowLongToBeat."""
    hltb = HowLongToBeat(session=session)
    results = await hltb.async_search(titulo, similarity_case_sensitive=False)

    if results is not None and len(results) > 0:
        print("resultado não nulo")
        best_element = max(results, key=lambda element: element.similarity)
        print(best_element.game_name)
        return best_element.json_content
    return None

async def main():
    jogos = []
    batch_size = 10  # Define o tamanho dos lotes

    async with aiohttp.ClientSession(timeout=aiohttp.ClientTimeout(total=30)) as session:
        for i in range(0, len(titulos), batch_size):
            batch = titulos[i:i + batch_size]

            # Executa as buscas em paralelo dentro do lote
            resultados = await asyncio.gather(
                *(buscar_jogo(titulo, session) for titulo in batch),
                return_exceptions=True
            )

            # Filtra os resultados válidos
            jogos.extend([r for r in resultados if r is not None and not isinstance(r, Exception)])
            print(jogos)

            # Aguarda 2 segundos entre os lotes para evitar bloqueios da API
            await asyncio.sleep(2)

    # Salva os dados em um arquivo Parquet
    
    df = pd.DataFrame(jogos)
    df.to_parquet("jrpgs_db.parquet", engine="pyarrow")
    print(df)
    return jogos

# Executa a função assíncrona
lista_jsons = await main()

[]
[]
[]


CancelledError: 

In [ ]:
import asyncio
import pandas as pd
from howlongtobeatpy import HowLongToBeat
import aiohttp
nao_conseguiu = []
async def buscar_jogo(titulo):
    """Busca informações sobre um jogo no HowLongToBeat."""
    try:
        hltb = HowLongToBeat()
        results = await hltb.async_search(titulo, similarity_case_sensitive=False)

        if results is not None and len(results) > 0:
            print("achou algo")
            best_element = max(results, key=lambda element: element.similarity)
            print(best_element.game_name)
            return best_element.json_content

    except Exception as e:
        print(f"Erro ao buscar '{titulo}': {e}")
        nao_conseguiu.append(titulo)
    
    return None

async def main():
    jogos = []
    batch_size = 5  # Número de requisições simultâneas por lote

    for i in range(0, len(titulos), batch_size):
        batch = titulos[i:i + batch_size]

        # Executa as buscas em paralelo dentro do lote
        resultados = await asyncio.gather(
            *(buscar_jogo(titulo) for titulo in batch),
            return_exceptions=True
        )

        # Filtra os resultados válidos e ignora exceções
        jogos.extend([r for r in resultados if r is not None and not isinstance(r, Exception)])

        # Aguarda 2 segundos antes de processar o próximo lote
        await asyncio.sleep(2)

    # Salva os dados em um arquivo Parquet
    df = pd.DataFrame(jogos)
    df.to_parquet("jrpgs_db.parquet", engine="pyarrow")
    print(df)

# Executa a função assíncrona
await main()


achou algo
.hack//G.U. Vol. 3: Redemption
achou algo
.hack//G.U. Vol. 1: Rebirth
achou algo
.hack//G.U. Vol. 2: Reminisce
achou algo
.hack//G.U. Last Recode
achou algo
.hack//Link
achou algo
7th Dragon 2020
achou algo
3D Dot Game Heroes
achou algo
13 Sentinels: Aegis Rim
achou algo
7th Dragon
achou algo
Tsukumogami
achou algo
A Healer Only Lives Twice
achou algo
7th Dragon III Code: VFD
achou algo
7th Dragon 2020-II
achou algo
A.W.: Phoenix Festa
achou algo
A Magical High School Girl
achou algo
Abarenbou Princess
achou algo
Advance Guardian Heroes
achou algo
ActRaiser
achou algo
ActRaiser Renaissance
achou algo
Advance Wars: Days of Ruin
achou algo
Adventure Bar Story
achou algo
Adventure Bar Story
achou algo
Adventure Academia: The Fractured Continent
achou algo
Advance Wars: Dual Strike
achou algo
Aegis of Earth: Protonovus Assault
achou algo
Adventures of Mana
achou algo
Aedis Eclipse: Generation of Chaos
achou algo
Adventures To Go!
achou algo
Adventure Labyrinth Story
achou algo
R

In [ ]:
nao_conseguiu

['.hack//G.U. Last Recode',
 '.hack//G.U. Vol. 1: Rebirth',
 '.hack//G.U. Vol. 2: Reminisce',
 '.hack//G.U. Vol. 3: Redemption',
 '.hack//Infection – Part 1',
 '.hack//Link',
 '.hack//Mutation – Part 2',
 '.hack//Outbreak – Part 3',
 '.hack//Quarantine – Part 4',
 '10101: “Will” the Starship',
 '13 Sentinels: Aegis Rim',
 '3 Heroes: Crystal Soul',
 '3D Dot Game Heroes',
 '7th Dragon',
 '7th Dragon 2020',
 '7th Dragon 2020-II',
 '7th Dragon III Code: VFD',
 '99 Spirits',
 'A Healer Only Lives Twice',
 'A King’s Tale: Final Fantasy XV',
 'A Magical High School Girl',
 'A Witch’s Tale',
 'A.W. Phoenix Festa',
 'Abarenbou Princess',
 'ACA Neo Geo: Crossed Swords',
 'Accel World vs. Sword Art Online: Millennium Twilight',
 'ActRaiser',
 'Actraiser Renaissance',
 'Advance Guardian Heroes',
 'Advance Wars: Days of Ruin',
 'Advance Wars: Dual Strike',
 'Advanced Dungeons &amp; Dragons: Dragons of Flame',
 'Adventure Academia: The Fractured Continent',
 'Adventure Bar Story',
 'Adventure Bar St

In [ ]:
with open("lista.txt", "w") as f:
    # Escrever a lista no arquivo
    f.write(str(nao_conseguiu))

In [ ]:
import ast

# Ler o conteúdo do arquivo .txt
with open("lista.txt", "r") as f:
    conteudo = f.read()

# Converter a string de volta para uma lista
minha_lista = ast.literal_eval(conteudo)

print(minha_lista)

""
